In [6]:
import csv
import json
from functools import reduce
from tqdm import tqdm

In [7]:
from rdflib import Graph, URIRef, Literal, RDF, ConjunctiveGraph
from rdflib import Namespace
from rdflib import Dataset
from rdflib.namespace import XSD

In [8]:
import pandas as pd
import numpy as np

In [9]:
usda_ingr_df = pd.read_excel('../data/input/ABBREV.xlsx',sheet_name='ABBREV')

In [10]:
usda_ingr_df.head()

,NDB_No,Shrt_Desc,Water_(g),Energ_Kcal,Protein_(g),Lipid_Tot_(g),Ash_(g),Carbohydrt_(g),Fiber_TD_(g),Sugar_Tot_(g),...,Vit_K_(µg),FA_Sat_(g),FA_Mono_(g),FA_Poly_(g),Cholestrl_(mg),GmWt_1,GmWt_Desc1,GmWt_2,GmWt_Desc2,Refuse_Pct
0,1001,"BUTTER,WITH SALT",15.87,717,0.85,81.11,2.11,0.06,0.0,0.06,...,7.0,51.368,21.021,3.043,215.0,5.0,"1 pat, (1"" sq, 1/3"" high)",14.2,1 tbsp,0.0
1,1002,"BUTTER,WHIPPED,WITH SALT",15.87,717,0.85,81.11,2.11,0.06,0.0,0.06,...,7.0,50.489,23.426,3.012,219.0,3.0,"1 pat, (1"" sq, 1/3"" high)",9.4,1 tbsp,0.0
2,1003,"BUTTER OIL,ANHYDROUS",0.24,876,0.28,99.48,0.00,0.00,0.0,0.00,...,8.6,61.924,28.732,3.694,256.0,12.0,1 tbsp,205.0,1 cup,0.0
3,1004,"CHEESE,BLUE",42.41,353,21.40,28.74,5.11,2.34,0.0,0.50,...,2.4,18.669,7.778,0.800,75.0,28.0,1 oz,17.0,1 cubic inch,0.0
4,1005,"CHEESE,BRICK",41.11,371,23.24,29.68,3.18,2.79,0.0,0.51,...,2.5,18.764,8.598,0.784,94.0,132.0,"1 cup, diced",113.0,"1 cup, shredded",0.0


In [14]:
datadict_df = pd.read_csv('../data/input/nutrient_list.csv')

In [15]:
datadict_df.head()

,Identifier,Label,Type,Unit,Position,RDV
0,http://www.w3id.org/foodkg/usda#description,Shrt Desc,Desc,NaN,2,0.0
1,http://www.w3id.org/foodkg/usda#water,Water (g),Water,g,3,0.0
2,http://www.w3id.org/foodkg/usda#energy,Energ Kcal,Energy,kcal,4,0.0
3,http://www.w3id.org/foodkg/usda#protein,Protein (g),Protein,g,5,50.0
4,http://www.w3id.org/foodkg/usda#lipid,Lipid Tot (g),Fat,g,6,78.0


In [16]:
usda_ingr_df.NDB_No= usda_ingr_df.NDB_No.astype(str).str.rjust(5,'0')

In [17]:
usda_ingr_df.head()

,NDB_No,Shrt_Desc,Water_(g),Energ_Kcal,Protein_(g),Lipid_Tot_(g),Ash_(g),Carbohydrt_(g),Fiber_TD_(g),Sugar_Tot_(g),...,Vit_K_(µg),FA_Sat_(g),FA_Mono_(g),FA_Poly_(g),Cholestrl_(mg),GmWt_1,GmWt_Desc1,GmWt_2,GmWt_Desc2,Refuse_Pct
0,01001,"BUTTER,WITH SALT",15.87,717,0.85,81.11,2.11,0.06,0.0,0.06,...,7.0,51.368,21.021,3.043,215.0,5.0,"1 pat, (1"" sq, 1/3"" high)",14.2,1 tbsp,0.0
1,01002,"BUTTER,WHIPPED,WITH SALT",15.87,717,0.85,81.11,2.11,0.06,0.0,0.06,...,7.0,50.489,23.426,3.012,219.0,3.0,"1 pat, (1"" sq, 1/3"" high)",9.4,1 tbsp,0.0
2,01003,"BUTTER OIL,ANHYDROUS",0.24,876,0.28,99.48,0.00,0.00,0.0,0.00,...,8.6,61.924,28.732,3.694,256.0,12.0,1 tbsp,205.0,1 cup,0.0
3,01004,"CHEESE,BLUE",42.41,353,21.40,28.74,5.11,2.34,0.0,0.50,...,2.4,18.669,7.778,0.800,75.0,28.0,1 oz,17.0,1 cubic inch,0.0
4,01005,"CHEESE,BRICK",41.11,371,23.24,29.68,3.18,2.79,0.0,0.51,...,2.5,18.764,8.598,0.784,94.0,132.0,"1 cup, diced",113.0,"1 cup, shredded",0.0


In [18]:
BASE= Namespace('http://idea.rpi.edu/heals/kb/usda#')
FOOD_P= Namespace('http://www.w3id.org/foodb/property/')
dataset = Dataset()
columns = usda_ingr_df.columns
for (index, series) in usda_ingr_df.iterrows():
    ing_no= str(series['NDB_No'])
    quantity = float(series['GmWt_1'])
    for i, row in datadict_df.iterrows():
        pos = row['Position']
        id_ = row['Identifier']
        uri = id_.replace('http://www.w3id.org/foodkg/usda#','https://w3id.org/foodkg/quality/high_')
        rdv = float(row['RDV'])
        
        col = columns[pos-1]
        #print (pos,col)
        if col in ['GmWt_1','GmWt_2','GmWt_Desc1','GmWt_Desc2','Shrt_Desc']:
            continue
        value = float(series[pos-1])
        if value  == 0.0 or str(value) == 'nan' or rdv ==0: continue
        portion_value = quantity*value/100
        if portion_value > rdv/3:
            dataset.add((BASE[ing_no],FOOD_P['hasQuality'], URIRef(uri)))

In [19]:
ingr_categ =pd.read_csv('../data/input/food_category.csv')
ingr_categ= ingr_categ.drop_duplicates(subset=['NDB_No'])

In [24]:
ingr_categ.head()

,NDB_No,Long_Desc,FdGrp_Cd,FdGrp_Desc
0,1001,"Butter, salted",100,Dairy and Egg Products
1,1002,"Butter, whipped, with salt",100,Dairy and Egg Products
2,1003,"Butter oil, anhydrous",100,Dairy and Egg Products
3,1004,"Cheese, blue",100,Dairy and Egg Products
4,1005,"Cheese, brick",100,Dairy and Egg Products


In [25]:
BASE= Namespace('http://idea.rpi.edu/heals/kb/usda#')
FOOD_P= Namespace('http://www.w3id.org/foodb/property/')
#dataset = Dataset()

for (index, series) in ingr_categ.iterrows():
    ing_no= str(series['NDB_No'])
    category = 'cat_'+str(series['FdGrp_Cd'])
    #print (ing_no, category)
    dataset.add((BASE[ing_no],FOOD_P['hasCategory'], BASE[category]))

In [26]:
dataset.serialize('../data/rdf/ingr_tags.nt', format='ntriples')